Fatima Seemab
291310


## Importing Libraries

In [112]:
%%capture
!python -m spacy download en


In [113]:
import os
import re
import time
import math
import random
import unicodedata

import numpy as np
import pandas as pd

from tqdm import tqdm

import spacy

from sklearn.model_selection import train_test_split

import torch
from torch import nn, optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [114]:
SEED = 28

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Preprocessing Data

In [115]:
data_df = pd.read_csv('lab_exam_dataset.txt', sep='\t', usecols=[0, 1])
data_df.columns = ['en', 'ur']
data_df.head()

en             ur
0    We won.    ہم جیت گئے۔
1   Beat it.     بھاگ جائو۔
2   Beat it.  دفعہ ہو جائو۔
3   We lost.    ہم ہار گئے۔
4  Good job!       پہت خوب۔

In [116]:
data_df.shape

(10502, 2)

In [117]:
seq_len_in = 20
seq_len_ot = 20

In [118]:
train_df, valid_df = train_test_split(data_df, test_size=0.2, shuffle=True, random_state=28)

train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)

print(train_df.shape)
print(valid_df.shape)

(8401, 2)
(2101, 2)


In [119]:
for i in range(len(train_df)-5, len(train_df)):
    print(f'ENGLISH:\n{train_df.iloc[i]["en"]},\nUrdu:\n{train_df.iloc[i]["ur"]}\n{"="*92}')

ENGLISH:
A lot has happened.,
Urdu:
کافی کچھ ہو گیا ہے۔
ENGLISH:
How long did you stay?,
Urdu:
کتنی دیر آپ نے قیام کیا تھا؟
ENGLISH:
How many teams are there?,
Urdu:
کتنی ٹیمیں ہیں؟
ENGLISH:
Spanish is his mother tongue.,
Urdu:
ہسپانوی اس کی مادری زبان ہے۔
ENGLISH:
I don't know her.,
Urdu:
میں اُس ( لڑکی ) کو نہیں جانتا ۔


## Creating Vocabulary

In [120]:
class Vocabulary:
    def __init__(self, freq_threshold=2, language='en', preprocessor=None, reverse=False):
        self.itos = {0: "<pad>", 1: "<sos>", 2: "<eos>", 3: "<unk>"}
        self.stoi = {"<pad>": 0, "<sos>": 1, "<eos>": 2, "<unk>": 3}
        self.tokenizer = spacy.blank(language)
        self.freq_threshold = freq_threshold
        self.preprocessor = preprocessor
        self.reverse = reverse

    def __len__(self):
        return len(self.itos)

    def tokenize(self, text):
        if self.reverse:
            return [token.text.lower() for token in self.tokenizer.tokenizer(text)][::-1]
        else:
            return [token.text.lower() for token in self.tokenizer.tokenizer(text)]

    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = len(self.itos)

        for sentence in sentence_list:
            if self.preprocessor:
                sentence = self.preprocessor(sentence)

            for word in self.tokenize(sentence):
                if word in frequencies:
                    frequencies[word] += 1
                else:
                    frequencies[word] = 1

                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self, text):
        tokenized_text = self.tokenize(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi["<unk>"]
            for token in tokenized_text
        ]

In [121]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = w.strip()
    return w

In [122]:
%%time
freq_threshold = 2
en_vocab = Vocabulary(freq_threshold=freq_threshold, language="en", preprocessor=preprocess_sentence, reverse=False)
de_vocab = Vocabulary(freq_threshold=freq_threshold, language="ur", reverse=False)
en_vocab.build_vocabulary(train_df["en"].tolist())
de_vocab.build_vocabulary(train_df["ur"].tolist())

CPU times: user 865 ms, sys: 7.49 ms, total: 873 ms
Wall time: 872 ms


In [123]:
class CustomTranslationDataset(Dataset):    
    def __init__(self, df, en_vocab, de_vocab):
        super().__init__()
        self.df = df
        self.en_vocab = en_vocab
        self.de_vocab = de_vocab
        
    def __len__(self):
        return len(self.df)
    
    def _get_numericalized(self, sentence, vocab):
      
        numericalized = [vocab.stoi["<sos>"]]
        numericalized.extend(vocab.numericalize(sentence))
        numericalized.append(vocab.stoi["<eos>"])
        return numericalized

    def __getitem__(self, index):
        en_numericalized = self._get_numericalized(self.df.iloc[index]["en"], self.en_vocab)
        de_numericalized = self._get_numericalized(self.df.iloc[index]["ur"], self.de_vocab)

        return torch.tensor(de_numericalized), torch.tensor(en_numericalized)

In [124]:
class CustomCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        src = [item[1] for item in batch]
        src = pad_sequence(src, batch_first=False, padding_value=self.pad_idx)
        
        targets = [item[0] for item in batch]
        targets = pad_sequence(targets, batch_first=False, padding_value=self.pad_idx)

        return src, targets

In [125]:
BATCH_SIZE = 128


train_dataset = CustomTranslationDataset(train_df, en_vocab, de_vocab)
valid_dataset = CustomTranslationDataset(valid_df, en_vocab, de_vocab)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=4,
    shuffle=False,
    collate_fn=CustomCollate(pad_idx=en_vocab.stoi["<pad>"])
)

valid_loader = DataLoader(
    dataset=valid_dataset,
    batch_size=BATCH_SIZE,
    num_workers=4,
    shuffle=False,
    collate_fn=CustomCollate(pad_idx=en_vocab.stoi["<pad>"])
)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [126]:
fun_de = np.vectorize(lambda x: de_vocab.itos[x])
fun_en = np.vectorize(lambda x: en_vocab.itos[x])

In [127]:
print(f"Unique tokens in source (ur) vocabulary: {len(de_vocab)}")
print(f"Unique tokens in target (eng) vocabulary: {len(en_vocab)}")

Unique tokens in source (ur) vocabulary: 1582
Unique tokens in target (eng) vocabulary: 1307


In [128]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Modeling

In [129]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hidden_dim, n_layers, dropout=0.2):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.gru = nn.GRU(emb_dim, hidden_dim, n_layers, bidirectional=True, dropout=0.0 if n_layers==1 else dropout)
        self.fc = nn.Linear(2*hidden_dim, hidden_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        x = self.embedding(x)
        x = self.dropout(x)
        outputs, hidden_state = self.gru(x)
       
        hidden_state = torch.tanh(self.fc(torch.cat((hidden_state[-2, :, :], hidden_state[-1, :, :]), dim=1)))
        return outputs, hidden_state

class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attention = nn.Linear(3*hidden_dim, hidden_dim)
        self.v = nn.Linear(hidden_dim, 1, bias=False)
    
    def forward(self, hidden_state, encoder_outputs):
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
       
        hidden_state = hidden_state.unsqueeze(1).repeat(1, src_len, 1)

        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        energy = torch.tanh(self.attention(torch.cat((hidden_state, encoder_outputs), dim = 2))) 
        
        attention = self.v(energy).squeeze(2)
        
        return F.softmax(attention, dim=1)

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hidden_dim, n_layers, attention, dropout=0.2):
        super().__init__()
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.attention = attention
        self.n_layers = n_layers
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.gru = nn.GRU(emb_dim+(hidden_dim*2), hidden_dim, n_layers, dropout=0.0 if n_layers==1 else dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim*3 + emb_dim, output_dim)
    
    def forward(self, input, hidden, encoder_outputs):
        input = input.unsqueeze(0)
        
        embedded = self.dropout(self.embedding(input))
        
        a = self.attention(hidden, encoder_outputs)
        a = a.unsqueeze(1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        weighted = torch.bmm(a, encoder_outputs)
        weighted = weighted.permute(1, 0, 2)
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        output, hidden = self.gru(rnn_input, hidden.unsqueeze(0))
        
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc(torch.cat((output, weighted, embedded), dim = 1))

        return prediction, hidden.squeeze(0)

class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        
        assert self.encoder.hidden_dim == decoder.hidden_dim
        assert self.encoder.n_layers == decoder.n_layers
    
    def forward(self, x, y, teacher_forcing_ratio=0.75):
        
        target_len = y.shape[0]
        batch_size = y.shape[1]
        target_vocab_size = self.decoder.output_dim  # Output dim
        
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        
        encoder_outputs, hidden_state = self.encoder(x)
        input = y[0,:]
        
        for t in range(1, target_len):
            output, hidden_state = self.decoder(input, hidden_state, encoder_outputs)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            pred = output.argmax(1)
            input = y[t] if teacher_force else pred
        
        return outputs

In [130]:
# Initialize all models
input_dim = len(en_vocab)
output_dim = len(de_vocab)
emb_dim = 256
hidden_dim = 512
n_layers = 1
dropout = 0.5

attention = Attention(hidden_dim)
encoder = Encoder(input_dim, emb_dim, hidden_dim, n_layers, dropout)
decoder = Decoder(output_dim, emb_dim, hidden_dim, n_layers, attention, dropout)
model = EncoderDecoder(encoder, decoder).to(device)

In [131]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

EncoderDecoder(
  (encoder): Encoder(
    (embedding): Embedding(1307, 256)
    (gru): GRU(256, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attention): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(1582, 256)
    (gru): GRU(1280, 512)
    (dropout): Dropout(p=0.5, inplace=False)
    (fc): Linear(in_features=1792, out_features=1582, bias=True)
  )
)

In [132]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 10,009,390 trainable parameters


In [133]:
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=en_vocab.stoi["<pad>"])

In [134]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    
    for i, batch in tqdm(enumerate(iterator), total=len(iterator), position=0, leave=True):
        src = batch[0].to(device)
        trg = batch[1].to(device)
        optimizer.zero_grad()
        output = model(src, trg)
        output_dim = output.shape[-1]
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [135]:
def evaluate(model, iterator, criterion):
    model.eval()    
    epoch_loss = 0

    with torch.no_grad():
        for i, batch in tqdm(enumerate(iterator), total=len(iterator), position=0, leave=True):
            src = batch[0].to(device)
            trg = batch[1].to(device)
            output = model(src, trg, 0)
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [136]:
def inference(model, sentence):
    model.eval()
    result, attentions = [], []

    with torch.no_grad():
        sentence = sentence.to(device)
        encoder_outputs, hidden_state = model.encoder(sentence)
        inp = torch.tensor([de_vocab.stoi["<sos>"]]).to(device)
        for t in range(1, seq_len_in):
            attention = model.decoder.attention(hidden_state, encoder_outputs)
            output, hidden_state = model.decoder(inp, hidden_state, encoder_outputs)
            pred = output.argmax(1)
            if pred == de_vocab.stoi["<eos>"]:
                break
            result.append(de_vocab.itos[pred.item()])
            inp = pred
            attentions.append(attention.cpu().detach().numpy().ravel())

    return " ".join(result), attentions

In [137]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [138]:
for sample_batch in valid_loader:
    break

In [145]:
N_EPOCHS = 20
CLIP = 1

best_valid_loss = float('inf')

sample_source = ' '.join([word for word in fun_en(sample_batch[0][:, 101]) if word not in ["<pad>", "<sos>", "<eos>"]])
sample_target = ' '.join([word for word in fun_de(sample_batch[1][:, 101]) if word not in ["<pad>", "<sos>", "<eos>"]])

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_loader, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best_model.pt')
    
    print(f'Epoch: {epoch+1:02}')
    print(f'\t Train Loss: {train_loss:.3f} ')
    print(f'\t Val. Loss: {valid_loss:.3f} ')
    print(f'\t Sample Source (English): {sample_source}')
    print(f'\t Sample Target (Urdu): {sample_target}')
    print(f'\t Generated: {inference(model, sample_batch[0][:, 101].reshape(-1, 1))[0]}\n')

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 17/17 [00:01<00:00,  9.36it/s]

Epoch: 01
	 Train Loss: 1.116 
	 Val. Loss: 2.012 
	 Sample Source (English): i <unk> <unk> lose !
	 Sample Target (Urdu): میں نہیں ہاروں گا ۔
	 Generated: میں ہل نہیں رہتا ۔




100%|██████████| 17/17 [00:01<00:00, 12.90it/s]

Epoch: 02
	 Train Loss: 0.664 
	 Val. Loss: 1.269 
	 Sample Source (English): i <unk> <unk> lose !
	 Sample Target (Urdu): میں نہیں ہاروں گا ۔
	 Generated: میں ہل نہیں ۔




100%|██████████| 17/17 [00:01<00:00, 12.50it/s]

...

Epoch: 18
	 Train Loss: 0.087 
	 Val. Loss: 0.413 
	 Sample Source (English): i <unk> <unk> lose !
	 Sample Target (Urdu): میں نہیں ہاروں گا ۔
	 Generated: میں نہیں ہاروں گا ۔




100%|██████████| 17/17 [00:01<00:00, 12.46it/s]

Epoch: 19
	 Train Loss: 0.090 
	 Val. Loss: 0.421 
	 Sample Source (English): i <unk> <unk> lose !
	 Sample Target (Urdu): میں نہیں ہاروں گا ۔
	 Generated: میں نہیں ہاروں گا ۔




100%|██████████| 17/17 [00:01<00:00, 12.41it/s]

Epoch: 20
	 Train Loss: 0.090 
	 Val. Loss: 0.428 
	 Sample Source (English): i <unk> <unk> lose !
	 Sample Target (Urdu): میں نہیں ہاروں گا ۔
	 Generated: میں نہیں ہاروں گا ۔



In [140]:
# Load the best model.
model_path = "./best_model.pt"
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

## Results

In [143]:
for idx in range(50):

    print(f'ACTUAL URDU: {" ".join([word for word in fun_de(sample_batch[1][:, idx]) if word not in ["<pad>", "<sos>", "<eos>"]])}')
    print(f'ACTUAL ENGLISH: {" ".join([word for word in fun_en(sample_batch[0][:, idx]) if word not in ["<pad>", "<sos>", "<eos>"]])}')
    print(f'GENERATED BY MODEL: {inference(model, sample_batch[0][:, idx].reshape(-1, 1))[0]}')
    print("="*92)

ACTUAL URDU: ٹام نے اسے نیچے گرا دیا ۔
ACTUAL ENGLISH: tom knocked him down .
GENERATED BY MODEL: ٹام نے مریم کو منہ رکھا ۔
ACTUAL URDU: ٹام زخمی ہے ۔
ACTUAL ENGLISH: tom <unk> hurt .
GENERATED BY MODEL: ٹام بیمار ہے ۔
ACTUAL URDU: میرے چاکلیٹ کا ڈبہ کہاں ہے ؟
ACTUAL ENGLISH: where <unk> my box of chocolates ?
GENERATED BY MODEL: آپ نے کہاں سے تمہارا ہے ؟
ACTUAL URDU: تمہاری امی بس آتی ہونگی ۔
ACTUAL ENGLISH: your mother will be back before long .
GENERATED BY MODEL: بولنے میں بس آتی ہونگی ۔
ACTUAL URDU: کیا آپ مجھے بتا سکتے ہے کہ میں اب کیا کروں ؟
ACTUAL ENGLISH: would you please tell me what i should do next ?
GENERATED BY MODEL: کیا آپ مجھے بتا سکتے ہو کہ میں اب آگے کیا کروں ؟
ACTUAL URDU: کیا تم گرین ٹی پی رہے ہو ؟
ACTUAL ENGLISH: are you drinking green tea ?
GENERATED BY MODEL: تم آپ گرین ٹی پی رہے ہو ؟
ACTUAL URDU: ٹام نے اسے نیچے گرا دیا ۔
ACTUAL ENGLISH: tom knocked him down .
GENERATED BY MODEL: ٹام نے مریم کو منہ رکھا ۔
ACTUAL URDU: بہت خوبصورت نام ہے تمہارا ۔
ACTUAL ENGLISH:

<ipython-input-143-b7d9dd542ef4>:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ot[idx,1]=" ".join([word for word in fun_en(sample_batch[0][:, idx]) if word not in ["<pad>", "<sos>", "<eos>"]])
<ipython-input-143-b7d9dd542ef4>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  ot[idx,2]= inference(model, sample_batch[0][:, idx].reshape(-1, 1))[0]
<ipython-input-143-b7d9dd542ef4>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  

ACTUAL URDU: میں غلطی سے آپ کو اپنا بھائی سمجھا ۔
ACTUAL ENGLISH: i mistook you for your brother .
GENERATED BY MODEL: میں آپ کو بھائی سے آپ کو جیت لیا ۔
ACTUAL URDU: میری بائسیکل کو مرمت کی ضرورت ہے ۔
ACTUAL ENGLISH: my bicycle is in need of repair .
GENERATED BY MODEL: وہ بائسیکل کو مرمت کی ضرورت ہے ۔
ACTUAL URDU: میرے ماموں ۱۹۸۳ میں میکسیکو گئے تھے اور کبھی واپس نہیں آئے ۔
ACTUAL ENGLISH: my uncle went to mexico in <unk> , never to come back .
GENERATED BY MODEL: میرے چاچا ۱۹۸۳ میں میکسیکو گئے تھے اور کبھی واپس نہیں آئے ۔
ACTUAL URDU: انہوں نے جاپانی شہریت حاصل کرلی ۔
ACTUAL ENGLISH: they became citizens of japan .
GENERATED BY MODEL: اس نے میری پیشکش دوڑے ۔
ACTUAL URDU: کیا آپ پہلی دفعہ جاپان آئے ہے ؟
ACTUAL ENGLISH: is this your first time in japan ?
GENERATED BY MODEL: کیا تم پہلی دفعہ جاپان آئی ہے ؟
ACTUAL URDU: گلاب کے پتے بہت نازک ہوتے ہے ۔
ACTUAL ENGLISH: a rose <unk> petals are very delicate .
GENERATED BY MODEL: وہ ایک بچہ نہیں تھا ۔
ACTUAL URDU: تم کیسے کر رہے ہو ؟
ACTUAL 